In [2]:
import numpy as np
import tensorflow as tf


In [3]:
ALPHABET = "abcdefghijklmnopqrstuvwxyz" # list of character
MAX_WORD_LEN = 15 # longest word our network can parse
ONE_HOT_SIZE = len(ALPHABET) * MAX_WORD_LEN # how long our one hot array should be

# get position of a char in alphabet
def getPos(char):
    if len(char) > 1:
        raise Exception("char should be a single character, length was: {}".format(len(char)))
    char = char.lower()
    if char not in ALPHABET:
        raise Exception("char should be in the alphabet, char was: {}".format(char))
    return ALPHABET.index(char)

# create a one hot array for value, length of length
def oneHot(val, length):
    arr = [0]*length
    arr[val] = 1
    return arr

# get one hot array for single char
def charOneHot(char):
    val = getPos(char)
    return oneHot(val, len(ALPHABET))

# get one hot arrays for each char in a string
def oneHotString(string):
    strLen = len(string)
    result = [0]*strLen
    for i in range(0, strLen):
        result[i] = charOneHot(string[i])
        
    return result

# transform a word to a one hot array 
def wordToInput(word):
    if len(word) > MAX_WORD_LEN:
        raise Exception("Word is too long, must be < 15 char, actual: {}".format(len(word)))
    result = [0]*ONE_HOT_SIZE
    index = 0
    for char in word:
        result[getPos(char) + index] = 1
        index = index + len(ALPHABET)
    return result
    
# great, now we can turn words into one hot arrays
# now we have to get and label training data

# functions for data wrangling

# returns true if word only contains letters in alphabet
def isAlpha(word):
    word = word.lower()
    for char in word:
        if char not in ALPHABET:
            return False
        
    return True
    

In [9]:
LANGUAGES = ["English", "German", "French"]
MIN_WORD_LEN = 3

# dictionary source: https://github.com/hermitdave/FrequencyWords

# data wrangling time, things we have to do:
# remove words shorter than 3(not required but it'll help) 
# remove words containing words outside of the english alphabet (this means no accents)
# remove words longer than 15 characters (or whatever we decide our maximum to be)
# the dataset I'm using also has frequency for each word, we don't need that

english = open("english_dict.txt", "r", encoding = "utf-8")

english_lines = english.readlines()
english_words = []
for line in english_lines:
    split = line.split(" ")
    word = split[0]
    if len(word) > MIN_WORD_LEN and len(word) < MAX_WORD_LEN and isAlpha(word):
        english_words.append(word)
        
        
german = open("german_dict.txt", "r", encoding = "utf-8")

german_lines = german.readlines()
german_words = []
for line in german_lines:
    split = line.split(" ")
    word = split[0]
    if len(word) > MIN_WORD_LEN and len(word) < MAX_WORD_LEN and isAlpha(word):
        german_words.append(word)
        
french = open("french_dict.txt", "r", encoding = "utf-8")

french_lines = french.readlines()
french_words = []
for line in french_lines:
    split = line.split(" ")
    word = split[0]
    if len(word) > MIN_WORD_LEN and len(word) < MAX_WORD_LEN and isAlpha(word):
        french_words.append(word)
        
print("Word counts before slice:")
print("English words: {}".format(len(english_words)))
print("German words: {}".format(len(german_words)))
print("French words: {} \n".format(len(french_words)))


#balancing
SAMPLES = 33000 # number of words for each language, should be less than smallest dictionary

english_words = english_words[0:SAMPLES]
german_words = german_words[0:SAMPLES]
french_words = french_words[0:SAMPLES]

print("Word counts after slice:")
print("English words: {}".format(len(english_words)))
print("German words: {}".format(len(german_words)))
print("French words: {}".format(len(french_words)))



Word counts before slice:
English words: 44295
German words: 39692
French words: 33678 

Word counts after slice:
English words: 33000
German words: 33000
French words: 33000


In [10]:
# time to label data
from sklearn.utils import shuffle


# oops I gotta actually use the one hot stuff
X_train = []
for word in english_words:
    X_train.append(wordToInput(word))

for word in german_words:
    X_train.append(wordToInput(word))
    
for word in french_words:
    X_train.append(wordToInput(word))

X_train = np.array(X_train)

y_train = []
y_train.extend([0]*SAMPLES)
y_train.extend([1]*SAMPLES)
y_train.extend([2]*SAMPLES)

y_train = tf.keras.utils.to_categorical(y_train)

y_train = np.array(y_train)

X_train, y_train = shuffle(X_train, y_train)
# data labeled in np array


In [11]:
import pickle

pickle_out = open("X.pickle", "wb")
pickle.dump(X_train, pickle_out)
pickle_out.close()

pickle_out = open("y.pickle", "wb")
pickle.dump(y_train, pickle_out)
pickle_out.close()

In [12]:
import pickle
X_train = pickle.load(open("X.pickle", "rb"))
y_train = pickle.load(open("y.pickle", "rb"))

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D

model = Sequential()
model.add(tf.keras.layers.Dense(ONE_HOT_SIZE))
model.add(Activation("relu"))
model.add(tf.keras.layers.Dense(256))
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(tf.keras.layers.Dense(256))
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(tf.keras.layers.Dense(3)) # output layer
model.add(Activation("softmax"))

model.compile(loss='categorical_crossentropy',
             optimizer = 'adam',
             metrics = ['accuracy'])

model.fit(X_train, y_train, batch_size = 32, validation_split = 0.1, epochs = 10)


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Train on 89100 samples, validate on 9900 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/10
89100/89100 [==============================] - 17s 191us/sample - loss: 0.8632 - acc: 0.5822 - val_loss: 0.7873 - val_acc: 0.6289
Epoch 2/10
89100/89100 [==============================] - 16s 182us/sample - loss: 0.7431 - acc: 0.6491 - val_loss: 0.7625 - val_acc: 0.6348
Epoch 3/10
89100/89100 [==============================] - 16s 184us/sample - loss: 0.6974 - acc: 0.6686 - val_loss: 0.7627 - val_acc: 0.6392
Epoch 4/10
89100/89100 [==============================] - 17s 186us/sample - loss: 0.6671 - acc: 0.6819 - val_loss: 0.7661 - val_acc: 0.6405
Epoch 5/10
89100/89100 [==============================] - 17s 188us/sample - loss: 0.6455 - acc: 0.6890 - val_loss: 0.7601 - val_acc: 0.6423
Epoch 6/10


In [14]:
model.save("English-German-French-v2.model")

In [15]:
model = tf.keras.models.load_model("English-German-French-v2.model")

In [19]:
word = "nuit"

word_vec = np.array(wordToInput(word)).reshape(-1, ONE_HOT_SIZE)
LANGUAGES[np.argmax(model.predict(word_vec))]



'French'